# Train darknet with custom dataset

this notebook demonstrates how to train darknet with following custom dataset

https://www.kaggle.com/andrewmvd/face-mask-detection

## download custom data

download kaggle api token (kaggle.json) from kaggle account and install kaggle utility. Seel this [link](https://www.kaggle.com/general/74235) for more information.

## prepare dataset format

convert COCO format to yolo format. below is a modified version of this [link](https://www.kaggle.com/rkuo2000/yolov5-facemask#Create-Dataset).

In [ ]:
import os
import numpy as np
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile

In [ ]:
def cord_converter(size, box):
    """
    convert xml annotation to darknet format coordinates
    :param size： [w,h]
    :param box: anchor box coordinates [upper-left x,uppler-left y,lower-right x, lower-right y]
    :return: converted [x,y,w,h]
    """
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def save_file(img_jpg_file_name, size, img_box):
    save_file_name = LABELS_ROOT + '/' + img_jpg_file_name + '.txt'
    print(save_file_name)
    file_path = open(save_file_name, "a+")
    for box in img_box:

        cls_num = classes.index(box[0])

        new_box = cord_converter(size, box[1:])

        file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

    file_path.flush()
    file_path.close()
    
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
    # print("img_name:", img_name)
    # print("image_info:(w,h,c)", img_w, img_h, img_c)
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        # print("box:(c,xmin,ymin,xmax,ymax)", cls_name, x1, y1, x2, y2)
        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2])
    # print(img_box)

    # test_dataset_box_feature(img_jpg_file_name, img_box)
    save_file(img_xml_file, [img_w, img_h], img_box)

In [ ]:
FILE_ROOT = "./dataset/"
IMAGE_PATH = FILE_ROOT + "images"  
ANNOTATIONS_PATH = FILE_ROOT + "annotations"
LABELS_ROOT = FILE_ROOT + "labels"

classes = ['with_mask', 'without_mask', 'mask_weared_incorrect']

In [ ]:
labels_dir = Path(LABELS_ROOT)
if not labels_dir.exists():
    print(f"Path {labels_dir} is not exit")
    os.makedirs(labels_dir)

In [ ]:
files = os.listdir(ANNOTATIONS_PATH)
for file in files:
    print("file name: ", file)
    file_xml = file.split(".")
    get_xml_data(ANNOTATIONS_PATH, file_xml[0])

## prepare training dataset and file

copy `labels.txt` to `images` dir and make a list of training images

In [ ]:
! cp $LABELS_ROOT/*.txt $IMAGE_PATH

In [ ]:
image_files = []
for filename in os.listdir(IMAGE_PATH):
    if filename.endswith(".png"):
        image_files.append(IMAGE_PATH + "/" + filename)
        
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()

## train with darknet

download pre-trained weights `darknet53.conv.74` for yolov3.

In [ ]:
! wget https://pjreddie.com/media/files/darknet53.conv.74

Train with yolov3. Don't execute this in notebook! but execute in a console. Use `nohup` command for long running training.

> assuming darknet build is located in `../darknet`. Training will take about 1 day to complete 6000 iterations in Azure NC6 (K80).


```
../darknet/darknet detector train yolov3-custom.data yolov3-train.cfg darknet53.conv.74 -dont_show
```

Training result will be save in the `backup` directory.